# 고차원 웹 애플리케이션 아키텍처

1. GET, POST, HEAD와 같은 HTTP 메소드들 각각에 대해 메소드의 목적과 기술적인 특징을 설명할 수 있어야 한다.  
    웹 브라우저 같은 client가 메소드를 사용하면 발생하는 작업들이 무엇인지, 그리고 HTTP 메소드에 대응하는 HttpServlet 메소드는 무엇인지 설명할 수 있어야 한다.  
2. servlet 생명 주기의 이벤트 순서와 목적을 설명할 수 있어야 한다.  
    (1) servlet 클래스 로딩  
    (2) servlet instantiation(인스턴스화)  
    (3) init 메서드 호출  
    (4) service 메서드 호출  
    (5) destory 메서드 호출  
3. 다음을 포함하는 web application의 file과 directory구조를 만들 줄 알아야 한다.  
    (1) static content  
    (2) JSP pages  
    (3) servlet classes  
    (4) deployment descriptor(배포 서술자)  
    (5) tag libraries  
    (6) JAR files  
    (7) Java class files  
    HTTP access로부터 파일 자원들을 어떻게 보호할지에 대해 설명할 수 있어야 한다.  
4. deployment descriptor(배포 서술자) 원소의 목적과 의미에 대해 설명할 수 있어야 한다.  
    : servlet instance, servlet name, servlet class, servlet initialization parameters, and servlet 매핑에 사용할 URL

## Container란?
servlet은 main() method가 없어서 Container라는 Java application의 지배를 받는다.  

tomcat은 Container의 한 가지 종류이다. Apache와 같은 web server application이 servlet에 대한 request를 받으면 server는 request를 servlet에게 바로 넘겨주는 것이 아니라, servlet을 관리하고 있는 Container에게 request를 준다. Container가 servlet에게 HTTP request와 response를 주고 doPost()나 doGet()같은 servlet 메소드를 호출한다.

### <span style="color:red">Container의 역할</span>
servlet을 관리하고 실행하는 것은 Container이다.
1. Communication support  
    container는 servlet이 web server와 통신할 수 있는 쉬운 방법을 제시한다.  
2. Lifecycle Management  
    Container는 클래스를 로딩하여 인스턴스화하고 servlet을 초기화하여 servlet메소드들을 호출하고 garbage collection이 servlet instance를 잘 읽을 수 있도록 만든다.  
3. Multithreading Support  
    Container는 들어오는 모든 servlet request에 대해 자동으로 Java thread를 만든다. 클라이언트의 요청에 따라 적절한 HTTP service 메서드가 끝나면, 스레드는 끝난다.  
4. Declarative Security(선언적인 보안)  
    Container를 사용하면 servlet class code안에 hard-coding할 필요 없이 XML deployment descriptor(배포 서술자)를 사용하여 보안 환경을 설정할 수 있다.  
5. JSP Support  
    Container가 JSP code를 실제 Java code로 변환해준다.  

### <span style="color:red">Container가 request를 다루는 방법</span>
1. user가 servlet으로 연결된 URL을 포함한 링크를 클릭한다.
2. container는 request가 servlet을 위한 것임을 확인하고 두 객체를 만든다.
    1) HttpServletResponse
    2) HttpServletRequest
3. container는 request상의 URL에 기초해서 올바른 servlet을 찾은뒤, 그 request를 위해 thread를 생성해서 request와 response 객체를 servlet 스레드로 넘긴다.
4. Container는 servlet의 service() 메소드를 호출한다. request의 종류에 따라 service() 메서드는 doGet() 또는 doPost() 메서드를 호출한다.
5. doGet()메소드를 호출했다면, 동적인 페이지를 생성하고 response 객체에 그 페이지를 실어 보낸다.
6. thread가 끝나면 container는 response 객체를 HTTP response로 전환하여 client에게 보낸다. 그리고 마지막으로 request와 response객체를 삭제한다.

### Container가 servlet을 찾는 방법
client로부터 날아온 request의 일부분에 있는 URL이 서버상의 특정한 servlet에 mapping된다.

컨테이너는 어떻게 servlet을 URL에 mapping할 수 있을까?
1. HTML page안에 Hardcoding하기
    - 장점: 간결함. 개발자는 URL 매핑을 직접적 제어할 수 있다. 
    - 단점: HTML 코드를 수정하지 않고 매핑을 변경할 수 없기 때문에 유연성이 부족하다.  
            프로젝트 규모가 커지면 여러 페이지에서 URL을 하드코딩하는 것은 유지보수가 어려워진다.  
            URL을 하드코딩하게 되면 구현의 세부사항을 노출시킬 수 있으므로 보안상의 문제가 발생한다.
2. Conainer vendor가 제공하는 mapping 도구 이용하기
    - 장점: application 코드로부터 매핑이 분리되어 구현된다. application code를 수정하지 않고 URL mapping을 쉬게 바꿀 수 있다. 
    - 단점: application을 특정 컨테이너 공급업체에 결합시켜 의존성이 강해진다. 도구에 따라 구현하는 프로세스가 복잡할 수 있다.
3. mapping 정보를 저장하기 위해 properties table(속성 테이블) 이용하기
    - 장점: 컨테이너 도구를 사용하는 것과 유사하게 응용 프로그램 코드 수정없이 URL 매핑을 동적으로 변경할 수 있다.  
            URL은 properties table에서 중앙 관리되어 유지보수가 쉽다. container vendor 도구보다 더 의존성이 낮다.
    - 단점: 속성 테이블을 설정하고 관리하는 것은 추가적인 노력을 필요로 한다. 수동으로 구현하는 경우 오류가 발생할 수 있다.  
            properties table을 사용하는 것은 추가적인 자원의 소비를 초래할 수 있다. 

## <span style="color:red">servlet code</span>
```java
public class Ch2Servlet extends HttpServlet {
    public void doGet(HttpServletRequest request, HttpServletResponse response) throws IOException {
        PrintWriter out = response.getWriter();
        java.util.Date today = new java.util.Date();
        out.prinln("<html> " +
                   "<body>" +
                   "<h1 style=text-align:center>" +
                   "HF's Chapter Servlet</h1>" +
                   "<br>" + today +
                   "</body>" +
                   "</html>");
    }
}
```

1. 거의 모든 servlet은 HttpServlet을 상속 받는다.  
2. 실생활에서는 servlet은 doGet()이나 doPost() 메서드만 override한다.
3. servlet이 container가 생성한 request와 response 객체에 대한 reference를 받는다.
4. container로부터 servlet이 얻은 response 객체로부터 PrinerWriter을 얻을 수 있다. response 객체에 HTML text를 쓰기 위해 PrintWrier를 사용하면 된다.


## Servlet
### servlet의 이름으로 사용할 수 있는 것
1. Client-known URL name(ex. register/registerMe)  
2. Deployer-known secret internal name(ex. EnrollServlet)  
3. Actual file name(ex. classes/registration/SignUpServlet.class)  

servlet이름을 다른 이름으로 mapping하면 application의 유연성과 보안을 향상시킨다.  

### <span style="color:red">Deployment Descriptor(배포 서술자)을 이용해서 URL을 servlet에 매핑하기</span>
web Container안에 있는 servlet을 배포하려면, Deployment Descriptor(DD)라고 불리는 XML 문서를 만들어서 Container에게 servlet과 JSP를 어떻게 실행할 것인지 알려줘야 한다.  
두 가지 XML 원소를 이용해서 URL을 servlet에 연결할 수 있다.  
\<servlet\>  
internal name과 fully-qualified(패키지 이름까지 포함한 완전한) class name을 mapping  
\<servlet-mapping\>  
client-known public URL name과 internal name을 mapping  

```java
<web-app xmlns="http://java.sun.com/xml/ns/j2ee"
         xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
         xsi:schemaLocation="http://java.sun.com/xml/ns/j2ee
         http://java.sun.com/xml/ns/j2ee/web-app_2_4.xsd"
         version="2.4">
    <servlet>
        <servlet-name>Inernal name 1</servlet-name>
        <servlet-class>foo.Servlet1</servlet-class>
    </servlet>

    <servlet>
        <servlet-name>Inernal name 2</servlet-name>
        <servlet-class>foo.Servlet2</servlet-class>
    </servlet>

    <servlet-mapping>
        <servlet-name>Chapter1 Servlet</servlet-name>
        <url-pattern>/Serv1</url-pattern>
    </servlet-mapping>
</web-app>
```
\<web-app\>: 이 웹 앱은 두 개의 servlet을 가지고 있다.  
\<servlet\> 원소는 container에게 어떤 class file이 어떤 특정한 web application에 속하는지 알려준다.  
\<servlet-name\> 원소는 < servlet> 원소를 특정한 < servlet-mapping>원소에 연결시켜준다. 이 이름은 DD에서만 사용된다.  
\<servlet-class\>에는 fully-qualified class name을 작성해야 한다. 하지만 .class 확장자는 붙이지 않는다.  
\<servlet-mapping\>원소를 Container가 request가 들어와서 실행할 때 사용하는 것이라고 생각한다면 '이번 request URL에는 어떤 servlet을 호출해야 될까?  
\<url-pattern\> client가 servlet에 접근할 때 보는 것이지만, 실제 servlet class의 이름은 아니다. wildcard(*)을 사용할 수 있다.

### 사이트 만들기
하나의 servlet은 하나의 responsibility(기능)만 가질 수 있도록 객체 지향적으로 설계를 했다.  
각각의 servlet은 결과를 client에게 HTML로 넘겨준다.  
println() 문장으로 HTML을 작성하다보니 지저분해져서 servlet이 비즈니스 로직을 수행하면, response에 대해 HTML을 하도록 요청(forward the request)을 JSP에게 넘긴다.  


## Deployment Descriptor(DD, 배포 서술자)
장점
* 이미 테스트된 소스 코드에 대한 수정을 최소화한다.
* 소스코드를 가지고 있지 않더라도 app의 목적에 맞게 수정할 수 있다.
* 코드를 테스트하거나 다시 컴파일하지 않고 application을 다른 자원에 연결시킬 수 있다.
* 접근 제어 목록(access control list)와 보안 역할(security role)과 같은 동적인 보안을 유지 관리하기 쉽게 한다.
* 프로그래머가 아닌 사람들이 web application을 수정하고 배포할 수 있다.


## MVC design pattern
Model * View * Controller(MVC)는 servlet로부터 비즈니스 로직을 분리하고, 비즈니스 로직을 재사용할 수 있는 평범한(plain old) Java class로 작성하여 Model에 넣는다.  

### MVC in the Servlet & JSP world
#### Model
실제 비즈니스 로직과 상태를 저장한다. 즉, Model은 상태를 getting하고, updating하는 규칙을 한다.  

#### View
presentation에 대한 책임을 지고있다. View는 Controller로부터 모델의 상태를 읽어온다. user input을 Controller에게 넘겨주는 역할도 한다.  

#### Controller
request로부터 user input을 가져와서 model에 대해 어떤 작업을 해야 하는지 알아낸다.   
Controller는 Model에게 업데이트해서 View에게 넘겨줄 새로운 모델 상태를 만들라고 한다.

### 적용하기
servlet에서 비즈니스 로직을 분리해서 Model을 표현할 수 있도록 각각의 servlet마다 일반적인 자바 클래스를 만들어야 한다.  
그러면 원래의 servlet은 Controller가 되고, 새로운 비즈니스 로직은 Model이 되며, JSP는 View가 된다.


## 핵심정리
1. Container는 web app 커뮤니케이션 지원, 생명주기 관리, 멀티 쓰레딩 지원, 선언적인 보안과 